In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


#tensorflow import
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

E:\anaconda\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, r"stock_data\index300_20_22_day.csv")

In [22]:
df = pd.read_csv(filename)

In [4]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600027.SH,20220825,5.61,5.78,5.51,5.71,5.49,0.22,4.0073,1052411.30,596661.861
1,600027.SH,20220824,5.68,5.77,5.49,5.49,5.67,-0.18,-3.1746,737333.57,411449.680
2,600027.SH,20220823,5.40,5.71,5.37,5.67,5.48,0.19,3.4672,1034137.80,580756.788
3,600027.SH,20220822,5.70,5.82,5.44,5.48,5.74,-0.26,-4.5296,1691234.19,949240.557
4,600027.SH,20220819,5.21,5.74,5.15,5.74,5.22,0.52,9.9617,1685692.64,939804.661


In [5]:
df.describe()

,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
count,2.647550e+05,264755.000000,264755.000000,264755.000000,264755.000000,264755.000000,264755.000000,264755.000000,2.647550e+05,2.647550e+05
mean,2.020943e+07,52.946048,54.185252,51.815468,53.014612,52.975329,0.039283,0.072446,4.650384e+05,8.753048e+05
std,7.704859e+03,119.308755,121.773700,117.042496,119.478963,119.388453,4.046699,3.126430,9.392410e+05,1.291714e+06
min,2.020010e+07,1.040000,1.070000,1.040000,1.050000,1.050000,-209.990000,-22.868700,2.579000e+02,4.906640e+02
25%,2.020091e+07,7.800000,7.920000,7.710000,7.810000,7.810000,-0.250000,-1.370150,9.108477e+04,2.013215e+05
50%,2.021051e+07,18.960000,19.300000,18.630000,18.970000,18.960000,0.000000,0.000000,2.238971e+05,4.421081e+05
75%,2.021123e+07,51.940000,53.075000,50.875000,51.950000,51.900000,0.235000,1.276800,4.983226e+05,9.909136e+05
max,2.022082e+07,2587.980000,2627.880000,2485.000000,2601.000000,2601.000000,228.980000,306.108600,4.114453e+07,4.796717e+07


In [13]:
stock_list = [x.sort_values("trade_date") for _,x in df.groupby('ts_code')]
#sort by trade_date

In [14]:
stock_list[0]

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
50685,000001.SZ,20200102,16.65,16.95,16.55,16.87,16.45,0.42,2.5532,1530231.87,2571196.482
50684,000001.SZ,20200103,16.94,17.31,16.92,17.18,16.87,0.31,1.8376,1116194.81,1914495.474
50683,000001.SZ,20200106,17.01,17.34,16.91,17.07,17.18,-0.11,-0.6403,862083.50,1477930.193
50682,000001.SZ,20200107,17.13,17.28,16.95,17.15,17.07,0.08,0.4687,728607.56,1247047.135
50681,000001.SZ,20200108,17.00,17.05,16.63,16.66,17.15,-0.49,-2.8571,847824.12,1423608.811
...,...,...,...,...,...,...,...,...,...,...,...
50047,000001.SZ,20220819,12.27,12.75,12.23,12.57,12.25,0.32,2.6122,2106417.24,2653471.090
50046,000001.SZ,20220822,12.58,12.68,12.41,12.50,12.57,-0.07,-0.5569,1138076.58,1423361.879
50045,000001.SZ,20220823,12.45,12.49,12.28,12.34,12.50,-0.16,-1.2800,921463.07,1137300.431
50044,000001.SZ,20220824,12.33,12.54,12.33,12.40,12.34,0.06,0.4862,1295375.33,1613309.589


In [8]:
'''
out = []

for ts_code in list(set(list(df['ts_code']))):
    out.append(df[df['ts_code']==ts_code])
'''

"\nout = []\n\nfor ts_code in list(set(list(df['ts_code']))):\n    out.append(df[df['ts_code']==ts_code])\n"

In [25]:
date_time = pd.to_datetime(stock_list[1].pop('trade_date'), format='%Y%m%d')

In [26]:
print(date_time)

37865   2020-01-02
37864   2020-01-03
37863   2020-01-06
37862   2020-01-07
37861   2020-01-08
           ...    
37227   2022-08-19
37226   2022-08-22
37225   2022-08-23
37224   2022-08-24
37223   2022-08-25
Name: trade_date, Length: 643, dtype: datetime64[ns]
